# Como usar o `Pipeline`

#### Tags

* Autor: AH Uyekita
* Data: 2019/03/22
* Post: [Como usar o `Pipeline`][post_url]

[post_url]: https://andersonuyekita.github.io/notebooks/blog/como-usar-o-pipepline/

#### Descrição

Esse é um arquivo de acompanhamento do _post_ de mesmo nome.

***

### Índice
- [1. Kernel](#1)
- [2. Importação dos _Packages_](#2)
- [3. _Datasets_](#3)
- [4. _Classifiers_](#4)
- [5. Uso do _Pipeline_](#5)
    - [5.1 Parte 1](#5.1)
    - [5.2 Parte 2](#5.2)
    - [5.3 Encapsulamento do _Pipeline_](#5.3)
- [6. _Pipeline para diversos Classifiers](#6)
- [7. Conclusões](#7)
- [8. Versões dos Packages](#8)

***

### 1. Kernel <a id='1'></a>

Foi usado o Python 3.7, conforme pode ser confirmado abaixo.

### 2. Importação dos _Packages_ básicos  <a id='2'></a>

In [1]:
# Importação dos packages básicos.
from sklearn import datasets
import pandas as pd
import numpy as np

# Só para evitar a aparição dos warnings.
import warnings
warnings.filterwarnings('ignore')

### 3. _Dataset_ <a id='3'></a>

Será usado como exemplo de dataset para a aplicação do `Pipeline` o banco de dados de câncer de mama.

In [2]:
# Carregando os dados de Câncer de mama.
cancer = datasets.load_breast_cancer()

O objeto `cancer` será um dicionário, dessa maneira vamos apenas carregar os dados que nos interessam.

In [3]:
# Criação do dataset features e vetor labels.
features = cancer.data
labels = cancer.target

### 4. _Classifier_ <a id='4'></a>

Esse exemplo terá como base a calibragem dos parâmetros do `Logistic Regression`.

In [4]:
# Importação do Logistic Regression.
from sklearn.linear_model import LogisticRegression

Vamos usar o `constructor` do `Logistic Regression` para criar um objeto chamado `clf`, ora doravante _classifier_.

In [5]:
# Uso do constructor do Logistic Regression para criar um classifier.
clf = LogisticRegression(random_state = 42,
                         solver = 'lbfgs',
                         multi_class = 'multinomial')

### 5. Uso do Pipeline <a id='5'></a>

A elucidação de como usar o Pipeline será exposto em duas partes:

1. Fazendo passo-a-passo e explicando cada etapa, e;
2. Encapsulando todas as etapas anteriores num Pipeline.

#### 5.1. Parte 1 <a id='5.1'></a>

Nessa etapa será feito cada etapa individualmente para um perfeito entendimento de cada uma delas e o seu papel no modelo.

Analisemos os dados presentes em _features_.

In [6]:
# Imprime as primeiras 5 linhas.
pd.DataFrame(features).head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Observa-se a presença de 30 _features_ com diferentes não _ranges_, isso pode ser um problema, pois o `Logistic Regression` utiliza a distância entre os valores para guiar o _Gradient Descent_, logo é imprescindível a mudança de escala das variáveis.

##### 5.1.1. Mudança de Escala

Para isso será utilizado o `MinMaxScaler` (leia a documentação [aqui][minmaxscaler_url]).

[minmaxscaler_url]: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [7]:
# Importação do MinMaxScaler.
from sklearn.preprocessing import MinMaxScaler

# Constructor para criar o objeto do MinMaxScaler.
scaler = MinMaxScaler()

# Treino.
scaler.fit(features)

# Transformação.
features_scaled = scaler.transform(features)

Vamos imprimir os resultados para verificar a mudança de escala.

In [8]:
# Imprimir somente as primeiras 5 linhas das features_scaled.
pd.DataFrame(features_scaled).head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,...,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,...,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
2,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,...,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
3,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,...,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
4,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,...,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595


##### 5.1.2. Diminuição de Dimensões

Devido à quantidade "elevada" de _features_, farei a reduação de 30 features para 10. Desta maneira, usarei o método de Componentes Principais para diminuir a dimensão do problema.

Leia a documentação do PCA [aqui][pca_url].

[pca_url]: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [9]:
# Importação do PCA.
from sklearn.decomposition import PCA

# Uso do constructor para criar o objeto do PCA.
pca = PCA(n_components = 10)

# Treino.
pca.fit(features_scaled)

# Tranformação.
features_scaled_pca = pca.transform(features_scaled)

Vamos imprimir os resultados para verificar a nova quantidade de colunas (_features_).

In [10]:
# Imprime as primeiras 5 linhas das features após PCA e scale.
pd.DataFrame(features_scaled_pca).head()

,0,1,2,3,4,5,6,7,8,9
0,1.387021,0.426895,-0.541703,0.048483,-0.072198,0.190817,0.236313,-0.039456,0.077590,0.155295
1,0.462308,-0.556947,-0.205175,-0.042830,0.016111,0.015604,0.043139,0.020644,-0.070639,-0.085284
2,0.954621,-0.109701,-0.147848,-0.001068,-0.033798,0.069062,-0.108166,0.007362,-0.059335,-0.073689
3,1.000816,1.525089,-0.053271,-0.207916,-0.219381,0.388007,0.194519,0.143499,0.176996,-0.140951
4,0.626828,-0.302471,-0.409336,0.238811,-0.002192,-0.157212,-0.063308,0.045931,0.002422,0.000545


##### 5.1.3. Criação de um Classifier

Com base no _classfier_ (`clf`) criado no item 4, vamos fazer previsões.

Primeiramente, vamos dividir o dataset em treino e teste para não incorrer em _overfitting_. O motivo de usar o `train_test_split` é simplesmente para simplificar o modelo e tornar o entendimento mais fácil.

Leia sobre o `train_test_spit` [aqui][train_test_split_url]

[train_test_split_url]: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [11]:
# Importação do StratifiedShuffleSplit.
from sklearn.model_selection import train_test_split

# Divisão dos features e labels em treino e teste.
features_train, features_test, labels_train, labels_test = train_test_split(features_scaled_pca,
                                                                            labels,
                                                                            test_size=0.33,
                                                                            random_state=42)

Agora que já foram criados os datasets de treino e teste, pode-se treinar e calcular as métricas.

In [12]:
# Treino usando o dataset de treino.
clf.fit(features_train, labels_train)

# Previsões usando o dataset de test.
pred = clf.predict(features_test)

# Importação do módulo para cálculo do Accuracy.
from sklearn.metrics import accuracy_score

# Avaliação.
accuracy_score(labels_test, pred)

0.9787234042553191

Após três etapas (5.1.1, 5.1.2 e 5.1.3) foi possível calcular o _accuracy_ do modelo. 

>Obs.: Não entraremos no mérito se os parâmetros da `Logistic Regression` são os melhores, pois esse não é o objetivo deste arquivo. A idéia aqui é analisar o _workflow_ da construção do modelo e como usar o `Pipeline` para automatizar parte disso.

#### 5.2. Parte 2 <a id='5.2'></a>

Após o pleno entendimento da Parte 1, onde foi feito passo-a-passo de cada etapa a Parte 2 tem como objetivo **encapsular** as três etapas elucidadas nos itens: 5.1.1, 5.1.2, e 5.1.3 em uma etapa só. Isso será feito usando o `Pipeline`.

In [13]:
# Importação do Pipeline
from sklearn.pipeline import Pipeline

A sequência de etapas até agora foi:

$$\text{Minhas Etapas = [Mudança de Escala, Diminuição de Dimensão, Previsão]}$$

O `Pipeline` necessita que seja criado uma lista de etapas, portanto será definido abaixo a lista de etapas com os seus respectivos "nomes" e **objetos**.

In [14]:
# Minha lista de etapas.
etapas_pipeline = [('scaler', scaler), # O scaler está definido na Etapa 5.1.1
                   ('pca'   , pca),    # O pca está definido na Etapa 5.1.2
                   ('clf'   , clf)]    # O clf está definido no item 4 e foi usado na Etapa 5.1.3

Todavia, pode-se substituir o **objeto** pelo **constructor**.

$$\text{('nome_da_etapa', constructor)}$$

Sendo assim uma generalização das nossas etapas.

In [15]:
# Minha lista de etapas.
etapas_pipeline = [('scaler', MinMaxScaler()),                                     # Etapa 5.1.1
                   ('pca'   , PCA(n_components = 10)),                             # Etapa 5.1.2
                   ('clf'   , LogisticRegression(random_state = 42,                # Etapa 5.1.3
                                                 solver = 'lbfgs',
                                                 multi_class = 'multinomial'))]

Por fim, tem-se que todas as etapas foram definidas e já é possível usar o _constructor_ do `Pipeline` para criar o objeto.

In [16]:
# Uso do constructor para criação do objeto do Pipeline.
pipe = Pipeline(etapas_pipeline)

Note que agora usaremos o objeto `pipe` como se ele fosse um classifier qualquer. Desta maneira, ele será treinado e depois usado para fazer previsões.

Analogamente à Etapa 1, será usado o simples `train_test_split` por ser o mais simples e de fácil entendimento. Observe que foi usado o **features original** sem modificações.

In [17]:
# Divisão dos features e labels em treino e teste.
features_train_2, features_test_2, labels_train_2, labels_test_2 = train_test_split(features,
                                                                                    labels,
                                                                                    test_size=0.33,
                                                                                    random_state=42)

Treina-se o `pipe` usando o par de treino (features de treino e labels de treino).

In [18]:
# Treino do pipe.
pipe.fit(features_train_2, labels_train_2);

Faz-se as previsões usando o `pipe` e o método `predict`.

In [19]:
# Fazendo previsões.
pred_2 = pipe.predict(features_test_2)

Agora podemos avaliar os resultaos e esperar que sejam iguais aos valores obtidos na Etapa 1.

In [20]:
    # Calculando o accuracy.
    accuracy_score(labels_test_2, pred_2)

0.9787234042553191

Observe que os resultados são iguais.

In [21]:
# Comparação entre os resultados das duas etapas. 
accuracy_score(labels_test_2, pred_2) == accuracy_score(labels_test, pred)

True

Note que o `Pipeline` aplica **ocultamente** os processos `fit` e `transform` das primeiras duas etapas, deixando apenas a parte de `fit` e `predict` do _classifier_. Logo, é um **requisito necessário** para que o `Pipeline` funcione que as etapas que antecedem o _classifier_ possuam esses dois métodos.

>**Não há como ter dois _classifiers_ como etapas do Pipeline. Só admite um _classifier_ e ele deve ser a última etapa.**

O motivo é simples: Os _classifiers_ geralmente não possuem o método `transform`, o que é um requisito do `Pipeline`.

#### 5.3 Encapsulamento do Pipeline <a id='5.3'></a>

O `Pipeline` pode ter o seu uso ampliado ao criar uma função, conforme exemplificado abaixo.

In [22]:
def meu_pipeline(n_components = 10, solver = 'lbgs', multi_class = 'multinomial',
                 features = features, labels = labels, test_size = 0.33):
    """
    Este é um exemplo bem simples de como encapsular o Pipeline numa função.
    Note que essa função oferece a possibilidade de teste de diversos parâmetros.
    """
    # Divisão dos features e labels em treino e teste.
    features_train_3, features_test_3, labels_train_3, labels_test_3 = train_test_split(features,
                                                                                        labels,
                                                                                        test_size = test_size,
                                                                                        random_state = 42)
    
    # Minha lista de etapas.
    etapas_pipeline = [('scaler', MinMaxScaler()),
                       ('pca'   , PCA(n_components = n_components)),
                       ('clf'   , LogisticRegression(random_state = 42,
                                                     solver = solver,
                                                     multi_class = multi_class))]
    
    # Treinando.
    pipe.fit(features_train_3, labels_train_3)
    
    # Prevendo.
    pred_3 = pipe.predict(features_test_3)
    
    # Comparação entre os resultados das duas etapas. 
    acc = accuracy_score(labels_test_3, pred_3)
    
    # Retorno do accuracy.
    return(acc)

Vamos testar a função usando os valores utilizados nas simulações anteriores e comparando o resultado final.

In [23]:
# Uso da função usando valores default.
meu_pipeline()

0.9787234042553191

Fazendo um teste com um número diferente de `test_size`.

In [24]:
# Calculando o Accuracy para um tamanho de teste de 20%.
meu_pipeline(test_size = 0.2)

0.9824561403508771

Portanto, o uso do `Pipeline` não altera em nada os resultados (desde que o algoritmo seja [**determinístico**][alg_deterministico]) e possui o benefício de proporcionar **menos** linhas de códigos.

[alg_deterministico]: https://pt.wikipedia.org/wiki/Algoritmo_determinístico

Os benefícios de um código mais enxuto (como esse da função):

* Estará menos suscetível a erro humano;
* Mais fácil de fazer a manutenção.

Em contra partida haverá:

* Maior abstração.

### 6. Pipeline para diversos _Classifiers_ (Técnicas "Avançadas") <a id='6'></a>

Uma maneira simples de utilizar vários _classifiers_ numa mesma estrutura de `Pipeline` é o uso de um segundo dicionário. A estrutura desse dicionário é a seguinte:

$$\text{meus_classifiers = \{'chave_1' : ('nome_1', constructor_1), } \\ \text{'chave_2' : ('nome_2', constructor_2)\}}$$

Vamos para um exemplo.

In [25]:
# Importação do Naïve Bayes.
from sklearn.naive_bayes import GaussianNB

# Dicionário de classifiers.
meus_classifiers = {'logit':('lr', LogisticRegression(random_state = 42,
                                                      solver = 'lbfgs',
                                                      multi_class = 'multinomial')),
                    'bayes':('gnb', GaussianNB())}

>Como isso funciona?

Ao definir a `chave` o dicionário retornará um _tuple_. Lembre-se que a lista de etapas do `Pipeline` deve ser composta de _tuples_.

Vamos ao teste! Usemos o Naïve Bayes cujo `key` é `bayes`.

In [26]:
# Imprime um tuple. Será o nome do objeto após o uso do constructor.
meus_classifiers['bayes']

('gnb', GaussianNB(priors=None, var_smoothing=1e-09))

A idéia é inserir esse dicionário dentro de outro que já conhecemos, o `etapas_pipeline`, que reproduzo abaixo.

In [27]:
# Minha lista de etapas.
etapas_pipeline = [('scaler', MinMaxScaler()),
                   ('pca'   , PCA(n_components = 10)),
                   ('clf'   , LogisticRegression(random_state = 42,
                                                 solver = 'lbfgs',
                                                 multi_class = 'multinomial'))]

Vamos substituir a última etapa da lista `etapa_pipeline` pelo nosso dicionário de _classifiers_ e usar `logit` como `key`.

In [28]:
# Minha lista de etapas com um dicionário aninhado.
etapas_pipeline_2 = [('scaler', MinMaxScaler()),
                     ('pca'   , PCA(n_components = 10)),
                      meus_classifiers['logit']]

Observe que se deve definir o `key`, caso contrário o Pipeline não conseguirá entender a lista de etapas, pois ele espera sempre receber uma lista composta por _tuples_.

Calculemos o _accuracy_ usando a lista `etapas_pipeline_2`.

In [29]:
# Uso do constructor para criação do objeto do Pipeline.
pipe_2 = Pipeline(etapas_pipeline_2)

# Treino do pipe.
pipe_2.fit(features_train_2, labels_train_2);

# Fazendo previsões.
pred_4 = pipe.predict(features_test_2)

# Calculando o accuracy.
accuracy_score(labels_test_2, pred_4)

0.9840425531914894

Note que se pode eliminar a lista de etapas e definí-la diretamente dentro do `Pipeline` conforme o exemplo abaixo.

In [30]:
# Uso do constructor para criação do objeto do Pipeline.
pipe_3 = Pipeline([('scaler', MinMaxScaler()),
                   ('pca'   , PCA(n_components = 10)),
                    meus_classifiers['bayes']])         # Estou usando o Naïve Bayes agora.

# Treino do pipe.
pipe_3.fit(features_train_2, labels_train_2);

# Fazendo previsões.
pred_5 = pipe_3.predict(features_test_2)

# Calculando o accuracy.
accuracy_score(labels_test_2, pred_5)

0.9202127659574468

Assim como foi feito no item 5.3 é possível o encapsulamento desse excerto de código acima numa função.

### 7. Conclusões <a id='7'></a>

O `Pipeline` pode automatizar grande parte do processo de calibragem do modelo e ser benéfico ao _workflow_, mas ele também pode ser usado de forma errada quando se adiciona etapas sem uma análise crítica. **Um exemplo é o que foi feito nesse arquivo**, a etapa de mudança de escala das _features_ foi feito sempre que o `Pipeline` foi executado.

Observe que convenientemente deve-se fazer a mudança de escala para otimizar o processo do _Gradient Descent_ dos algoritmos que são baseados em distância, logo de forma geral é muito aconselhável fazer a mudança de escala para todos os algoritmos, pois não há nenhuma desvantagem em mudar a escala das _features_. Portanto, a remoção do `MinMaxScaler` da lista de etapas e executando ele antes do `Pipeline` resultará num algoritmo mais eficiente, pois ele será executado uma vez só.

Já para o caso do `PCA` é conveniente que se deixe dentro do `Pipeline`, pois dará a oportunidade de variarmos a quantidade de componentes principais (`n_components`).

Por fim, tenha em mente que para _datasets_ pequenos o tempo de execução não é algo a se preocupar, mas em casos reais onde o tempo é um fator limitante, eliminar repetições desnecessárias é fundamental para otimizar os gastos dos seus recursos (tempo, dinheiro, esforço, paciência, etc.).


### 8. Versões <a id='8'></a>

Usei o [sinfo][sinfo_url] para imprimir as versões de cada módulo e package utilizado neste _Jupyter Notebook_.

[sinfo_url]: https://gitlab.com/joelostblom/sinfo

```
# No terminal/Prompt.
pip install sinfo
```

In [31]:
# Importa o sinfo.
import sinfo

# Imprime as versões dos Packages.
sinfo.sinfo()

-----
numpy     	1.15.4
pandas    	0.23.4
sklearn   	0.20.2
-----
IPython   	7.2.0
jupyter_client	5.2.4
jupyter_core	4.4.0
notebook  	5.7.4
-----
Python 3.7.1 (v3.7.1:260ec2c36a, Oct 20 2018, 14:05:16) [MSC v.1915 32 bit (Intel)]
Windows-10-10.0.17763-SP0
4 logical CPU cores, Intel64 Family 6 Model 61 Stepping 4, GenuineIntel
-----
Session information updated at 2019-03-23 04:18
